## Overview
This notebook walks through creating, setting up and training a Deep Markov Model on a synthetic dataset

In [1]:
#General Purpose Imports
import numpy as np
import pandas as pd
import glob, os, sys, time
sys.path.append('/Users/shiyunqiu/dmm/')
from utils.misc import getConfigFile, readPickle, displayTime

/Users/shiyunqiu/miniconda3/envs/myenv/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## A] Data
* The polyphonic music code (which can be run from the `expt` folder) represents an example of running on binary data
* There is code to create a synthetic dataset in `dmm_data/load.py`, we will load that dataset here (it will be created the first time this function is called)

In [4]:
#Import load function to load synthetic data
# from dmm_data.load import load
# dataset = load('synthetic')
# print type(dataset), dataset.keys()

Creating linear matrices
Creating linear matrices
Saved  1  objects
Saving...
<type 'dict'> ['test', 'dim_observations', 'train', 'valid', 'data_type']


### a) Dataset format
The dataset's are expected to be in a specific format:
* The type of the variable `dataset` is `dict`
* The keys of this dataset correspond to parameters used by the model as well as raw data
* `dim_observations` is of type `int` and it corresponds to the dimensionality of the raw data at each point in time
* `data_type` can be `binary` or `real`
* `train`,`valid` and `test` are dictionaries that house the pre-split data. `dataset['train']` contains three items: 
    * `dataset['train']['tensor']` is a 3D tensor where the dimensions correspond to `Nsamples x T x dim_observations`
    * `dataset['train']['mask']` is a 2D matrix where the dimensions correspond to `Nsamples x T`. Each entry in this matrix corresponds to whether a sample was observed at that point in time. This is used internally by the model to model variable length sequences. 
    * `dataset['train']['tensor_Z']` is not used by the model. It corresponds to the states of the **true** underlying latent variables that created this data

In [5]:
# print 'Dimensionality of the observations: ', dataset['dim_observations']
# print 'Data type of features:', dataset['data_type']
# for dtype in ['train','valid','test']:
#     print 'dtype: ',dtype, ' type(dataset[dtype]): ',type(dataset[dtype])
#     print [(k,type(dataset[dtype][k]), dataset[dtype][k].shape) for k in dataset[dtype]]
#     print '--------\n'

Dimensionality of the observations:  3
Data type of features: real
dtype:  train  type(dataset[dtype]):  <type 'dict'>
[('tensor_Z', <type 'numpy.ndarray'>, (10000, 10, 3)), ('mask', <type 'numpy.ndarray'>, (10000, 10)), ('tensor', <type 'numpy.ndarray'>, (10000, 10, 3))]
--------

dtype:  valid  type(dataset[dtype]):  <type 'dict'>
[('tensor_Z', <type 'numpy.ndarray'>, (1000, 10, 3)), ('mask', <type 'numpy.ndarray'>, (1000, 10)), ('tensor', <type 'numpy.ndarray'>, (1000, 10, 3))]
--------

dtype:  test  type(dataset[dtype]):  <type 'dict'>
[('tensor_Z', <type 'numpy.ndarray'>, (1000, 10, 3)), ('mask', <type 'numpy.ndarray'>, (1000, 10)), ('tensor', <type 'numpy.ndarray'>, (1000, 10, 3))]
--------



### b) Creating your own dataset
* When creating your own data, it should have the following structure: 
    * type: dict
        * `dim_observations`: int
        * `data_type`: `binary` or `real`
        * `train`, `valid`, `test` must be dictionaries with the following keys: 
            * `tensor` : Raw data as a 3D tensor with dimensions `Nsamples x T x dim_observations`
            * `mask`   : Mask for the raw data as a 2D matrix with dimensions `Nsamples x T`
* Now that we have the dataset in the desired format, lets look at setting up the model, we'll first load the necessary files to build the model

In [2]:
import pandas as pd
df = pd.read_csv('/Users/shiyunqiu/AM231_Project/data/bitcoin_price.csv')
df = df[::-1]
df = df.Close.values.reshape(-1,1)

In [3]:
df.shape

(1760, 1)

In [4]:
train_size = int(len(df) * 0.8)
valid_size = int(len(df) * 0.1)
test_size = len(df) - train_size - valid_size
# train = df[0:train_size, :]
# valid = df[train_size:train_size+valid_size, :]
# test = df[train_size+valid_size:len(dataset), :]

In [5]:
def generate_sample(T, nsample, start_indx, end_indx):
    sample = []
    idx = np.random.randint(start_indx, end_indx, nsample, 'int')
    for i, val in enumerate(idx):
        piece = df[val:val+10, :]
        sample.append(piece)
    return np.array(sample)

In [13]:
# Train: T=10, 1000 samples
train = generate_sample(10, 1000, 0, train_size-10)

# Valid: T=10, 100 samples
valid = generate_sample(10, 100, train_size, train_size+valid_size-10)

# Test: T=10, 100 samples
test = generate_sample(10, 100, train_size+valid_size, len(df)-10)

In [15]:
train_dict = {}
train_dict['tensor'] = train
train_dict['mask'] = np.ones((1000, 10))

valid_dict = {}
valid_dict['tensor'] = valid
valid_dict['mask'] = np.ones((100, 10))

test_dict = {}
test_dict['tensor'] = test
test_dict['mask'] = np.ones((100, 10))

In [16]:
dataset = {}
dataset['dim_observations'] = 1
dataset['data_type'] = 'real'
dataset['train'] = train_dict
dataset['valid'] = valid_dict
dataset['test'] = test_dict

In [17]:
print 'Dimensionality of the observations: ', dataset['dim_observations']
print 'Data type of features:', dataset['data_type']
for dtype in ['train','valid','test']:
    print 'dtype: ',dtype, ' type(dataset[dtype]): ',type(dataset[dtype])
    print [(k,type(dataset[dtype][k]), dataset[dtype][k].shape) for k in dataset[dtype]]
    print '--------\n'

Dimensionality of the observations:  1
Data type of features: real
dtype:  train  type(dataset[dtype]):  <type 'dict'>
[('mask', <type 'numpy.ndarray'>, (1000, 10)), ('tensor', <type 'numpy.ndarray'>, (1000, 10, 1))]
--------

dtype:  valid  type(dataset[dtype]):  <type 'dict'>
[('mask', <type 'numpy.ndarray'>, (100, 10)), ('tensor', <type 'numpy.ndarray'>, (100, 10, 1))]
--------

dtype:  test  type(dataset[dtype]):  <type 'dict'>
[('mask', <type 'numpy.ndarray'>, (100, 10)), ('tensor', <type 'numpy.ndarray'>, (100, 10, 1))]
--------



In [18]:
start_time = time.time()
from   model_th.dmm import DMM
import model_th.learning as DMM_learn
import model_th.evaluate as DMM_evaluate
displayTime('importing DMM',start_time, time.time())

		< importing DMM > took  2.15572285652   seconds


## B] Model Hyperparameters
* To setup the model, we need to specify the hyperparameters
* Normally, if you were running from a script, you would run the following: `from parse_args import params` (e.g `expt/train.py` within the script.
* This lets you specify hyperparameters for the model via the command line. See the shell scripts in the folder `expt/` for an example of this. 
* Since we're in Ipython, we'll reload a saved version of `params` and see what the default values currently are. To know more about how the choices of hyperparameters affect the model, you can run `python parse_args.py -h` in the main directory.
* The `unique_id` is created based on the default parameters or those specified via the commend line.
* The parameters (`data_type` and `dim_observations`) of the dataset need to be incorporated into `params` for the model to be able to setup the weights appropriately

In [19]:
params = readPickle('../default.pkl')[0]
for k in params:
    print k, '\t',params[k]
params['data_type'] = dataset['data_type']
params['dim_observations'] = dataset['dim_observations']

Read  1  objects
dataset 	mm
epochs 	2000
seed 	1
init_weight 	0.1
dim_stochastic 	100
expt_name 	uid
reg_value 	0.05
reloadFile 	./NOSUCHFILE
reg_spec 	_
dim_hidden 	200
lr 	0.0008
reg_type 	l2
init_scheme 	uniform
optimizer 	adam
use_generative_prior 	approx
maxout_stride 	4
batch_size 	20
savedir 	./chkpt
forget_bias 	-5.0
inference_model 	R
emission_layers 	2
savefreq 	10
rnn_cell 	lstm
rnn_size 	600
paramFile 	./NOSUCHFILE
nonlinearity 	relu
rnn_dropout 	0.1
transition_layers 	2
anneal_rate 	2.0
debug 	False
validate_only 	False
transition_type 	mlp
unique_id 	DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid
leaky_param 	0.0


## C] Building the DMM

In [20]:
#The dataset is small, lets change some of the default parameters and the unique ID
params['dim_stochastic'] = 2
params['dim_hidden']     = 20
params['rnn_size']       = 40
params['epochs']         = 40
params['batch_size']     = 200
params['unique_id'] = params['unique_id'].replace('ds-100','ds-2').replace('dh-200','dh-20').replace('rs-600','rs-40')
params['unique_id'] = params['unique_id'].replace('ep-2000','ep-40').replace('bs-20','bs-200')

#Create a temporary directory to save checkpoints
params['savedir']   = params['savedir']+'-ipython/'
os.system('mkdir -p '+params['savedir'])

#Specify the file where `params` corresponding for this choice of model and data will be saved
pfile= params['savedir']+'/'+params['unique_id']+'-config.pkl'

print 'Checkpoint prefix: ', pfile
dmm  = DMM(params, paramFile = pfile)

Checkpoint prefix:  ./chkpt-ipython//DMM_lr-0_0008-dh-20-ds-2-nl-relu-bs-200-ep-40-rs-40-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-config.pkl
	<<Sampling biases for LSTM from exponential distribution>>
	<<Nparameters: 14410>>
	<<WARNING: lr will not differentiated with respect to>>
	<<WARNING: anneal will not differentiated with respect to>>
	<<WARNING: update_ctr will not differentiated with respect to>>
	<<Anneal = 1 in 2.0 param. updates>>
	<<Building with RNN dropout:0.1>>
	<<In _LSTM_RNN_layer with dropout 0.1000>>
	<<Modifying : [q_W_input_0,q_b_input_0,W_lstm_r,b_lstm_r,U_lstm_r,q_W_st,q_b_st,q_W_mu,q_b_mu,q_W_cov,q_b_cov,p_trans_W_0,p_trans_b_0,p_trans_W_1,p_trans_b_1,p_trans_W_mu,p_trans_b_mu,p_trans_W_cov,p_trans_b_cov,p_emis_W_0,p_emis_b_0,p_emis_W_1,p_emis_b_1,p_emis_W_out,p_emis_b_out]>>
<< Reg:(l2) Reg. Val:(0.05) Reg. Spec.:(_)>>
<<<<<< Adding l2 regularization for q_W_input_0 >>>>>>
<<<<<< Adding l2 regularization for q_b_input_0 >>>>>>
<<<<<< Adding l2 re

## D] Parameter Estimation

In [21]:
#savef specifies the prefix for the checkpoints - we'll use the same save directory as before 
savef    = os.path.join(params['savedir'],params['unique_id'])
savedata = DMM_learn.learn(dmm, dataset['train'], epoch_start =0 ,
                                epoch_end = params['epochs'],
                                batch_size = 200,
                                savefreq   = params['savefreq'],
                                savefile   = savef,
                                dataset_eval=dataset['valid'],
                                shuffle    = True )

	<<Original dim: [3 5 1],[3 5]>>
	<<New dim: [1000   10    1],[1000   10]>>
	<<Bnum: 0, Batch Bound: 121066.0698, |w|: 22.2577, |dw|: 1.0000, |w_opt|: 0.0000>>
	<<-veCLL:242132138.8413, KL:82.8648, anneal:0.0100>>
	<<(Ep 0) Bound: 121748.4526 [Took 0.3633 seconds] >>
	<<Saving at epoch 0>>
	<<Saved model (./chkpt-ipython/DMM_lr-0_0008-dh-20-ds-2-nl-relu-bs-200-ep-40-rs-40-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP0-params) 
		 opt (./chkpt-ipython/DMM_lr-0_0008-dh-20-ds-2-nl-relu-bs-200-ep-40-rs-40-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP0-optParams) weights>>
	<<Original dim: [1000   10    1],[1000   10]>>
	<<New dim: [100  10   1],[100  10]>>
	<<(Evaluate) Validation Bound: 2477875.8306 [Took 0.0122 seconds]>>
	<<Original dim: [100  10   1],[100  10]>>
	<<New dim: [1000   10    1],[1000   10]>>
	<<Bnum: 0, Batch Bound: 123485.7700, |w|: 22.2575, |dw|: 1.0000, |w_opt|: 0.4095>>
	<<-veCLL:246971439.8253, KL:100.0808, anneal:1.0000>>
	<<(Ep 1) Bound: 12

	<<Bnum: 0, Batch Bound: 13.0916, |w|: 22.3120, |dw|: 1.0000, |w_opt|: 0.9776>>
	<<-veCLL:19480.5608, KL:6702.7217, anneal:1.0000>>
	<<(Ep 31) Bound: 13.2474 [Took 0.5553 seconds] >>
	<<Bnum: 0, Batch Bound: 13.2991, |w|: 22.0427, |dw|: 1.0000, |w_opt|: 0.7308>>
	<<-veCLL:20296.8739, KL:6301.4260, anneal:1.0000>>
	<<(Ep 32) Bound: 12.8589 [Took 0.5022 seconds] >>
	<<Bnum: 0, Batch Bound: 12.4563, |w|: 21.7881, |dw|: 1.0000, |w_opt|: 0.5485>>
	<<-veCLL:19326.7538, KL:5585.7974, anneal:1.0000>>
	<<(Ep 33) Bound: 12.9108 [Took 0.4604 seconds] >>
	<<Bnum: 0, Batch Bound: 11.2107, |w|: 21.6209, |dw|: 1.0000, |w_opt|: 0.5139>>
	<<-veCLL:18331.9170, KL:4089.5183, anneal:1.0000>>
	<<(Ep 34) Bound: 11.7725 [Took 0.5826 seconds] >>
	<<Bnum: 0, Batch Bound: 11.6001, |w|: 21.5315, |dw|: 1.0000, |w_opt|: 0.4322>>
	<<-veCLL:19555.7342, KL:3644.5512, anneal:1.0000>>
	<<(Ep 35) Bound: 10.9250 [Took 0.4263 seconds] >>
	<<Bnum: 0, Batch Bound: 10.5353, |w|: 21.4828, |dw|: 1.0000, |w_opt|: 0.3986>>
	<<-v